In [ ]:
#Plot Bot

# Dependencies
import tweepy
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import json

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Setup Tweepy API Authentication
from config import (consumer_key, consumer_secret, access_token, access_token_secret)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
def vader_tweet(tweet):
    '''returns vader polarity scores with the addition of a overall polarity score'''

    results = analyzer.polarity_scores(tweet)
    return results

In [ ]:
def tweet_sentiments(user):

    # Counter
    counter = 1

    # Variables for holding sentiments
    sentiments = []

    # Loop through 25 pages of tweets (total 100 tweets)
    for x in range(5):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user)

        # Loop through all tweets 
        for tweet in public_tweets:
            print("Analyzing tweet: \n")
            print("---" * 30)
            pprint(tweet['text'])
            
            # Run Vader Analysis on each tweet
            results = vader_tweet(tweet['text'])
            tweets_ago = counter
            sentiments.append(results)
            # Add to counter 
            counter = counter + 1
    polarities = list()
    positivity_list = list()
    negativity_list = list()
    for record in sentiments:
        polarities.append(record['compound'])
        positivity_list.append(record['pos'])
        negativity_list.append(record['neg'])
    
    final_json = json.dumps({ "user":user, 
                              "polarity":polarities,
                              "neg":negativity_list,
                              "pos":positivity_list
                             })
    df = pd.read_json(final_json)
    return df

In [ ]:
def tweet_plot(sentiments, user):
    
    #plot out polarities positive and negative
    plt.rcParams["figure.figsize"] = (10,8)
    
    pos = sentiments.pos.values
    neg = sentiments.neg.values
    polarity = sentiments.polarity.values
    
    ymax = max(np.append(pos, neg)) + .5
   
    
    f, (ax1,ax2, ax3) = plt.subplots(3, 1, sharey=True, sharex=True)
    ax1.set_ylim(ymax=ymax, ymin=-1)
    x_axis = np.arange(0,len(sentiments))
    #sns.kdeplot(sentiments.pos.values, shade=True, color="forestgreen", ax=ax1)
    #sns.kdeplot(sentiments.neg.values,shade=True, color='coral',  ax=ax2)
    ax1.plot(x_axis, pos, '-', color='coral', linewidth=4, marker="o")
    ax2.plot(x_axis, neg, '-', color='forestgreen', linewidth=4, marker="o")
    ax3.plot(x_axis, polarity, '-', color='skyblue', linewidth=4, marker="o")
    #  Incorporate the other graph properties
    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    ax1.set_title("Sentiment Analysis of Tweets ({}) for {}".format(now, user))
    
    #draw horizontal line that seperates positive and negative values
    ax1.axhline(y=0, xmin=0, xmax=max(x_axis),color="red")
    ax2.axhline(y=0, xmin=0, xmax=max(x_axis),color="red")
    ax3.axhline(y=0, xmin=0, xmax=max(x_axis),color="red")
    #labels
    ax3.set_ylabel("Tweet Polarity")
    ax2.set_ylabel("Tweet Negativity")
    ax1.set_ylabel("Tweet Positivity")
    ax3.set_xlabel("Tweets Ago")
    plt.savefig(f"{user}.png")
#tweet_plot(sentiments, sentiments['user'][0])

In [ ]:
def mention_check(starting_id):
    ''' activates when a twitter user tweets @PlotMood and then returns a graph of polarities
        
        old_mentions: previous iterations of tweets that you want to exclude from the plots
    
    '''
    
    # Target Term
    target_term = "@MoodPlot"
    
    # Get the last 10 tweets
    public_tweets = api.search(target_term, count=10, result_type="recent", since_id=starting_id)
    
    output_dict = list()
    
    # Loop through all public_tweets
    for tweet in public_tweets["statuses"]:
        
        # Get ID and Author of most recent tweet
        
        tweet_id = tweet["id"]
        tweet_author = tweet["user"]["screen_name"]
        tweet_text = tweet["text"]
        user_mentions = set()
        for user in tweet['entities']['user_mentions']:
            if user in old_mentions:
                pass
            else:
                output_dict.append({'id':tweet_id, 'user':user['screen_name']})
        
        

    # Print success message
    print("checked successfully, sir!")
    return output_dict

start = None
mention_check(start)
            

In [ ]:
# Set timer to run 5 minutes every hour
import time 
t_end = time.time() + 3600 * 24

start = None
while(time.time() < t_end):
    records = mention_check(start)

    
    
    print("Activating Twitter bot")

    for record in records:
        print("running anaylsis for: " + record['user'])
        if record['user'] == 'MoodPlot':
            pass
        else:
            user_sentiments = tweet_sentiments(record['user'])
            tweet_plot(user_sentiments, record['user'])

            #read tweet_plot output
            #file = open(f"{user}.png", 'rb')
            #data = file.read()

            #upload media to twitter
            res = api.media_upload(f"{record['user']}.png")
            print(res)
            media_id = res['media_id_string']
            print(media_id)
            api.update_status(status=f"Here's plot analysis for {record['user']}",in_reply_to_status_id=record['id'], media_ids=[media_id])

            #wait thirty seconds before sending out tweet   
                              time.sleep(30)
    start = record[-1]['id']